# <span style = 'color:#057296;font-family:helvetica'> Introduction To Recommender System

In [4]:
import findspark

In [5]:
findspark.init('/home/chandan/spark-3.2.4-bin-hadoop3.2')

In [6]:
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession.builder.appName('rs').getOrCreate()

### The two most common types of recommender systems are `Content-Based` and `Collaborative Filtering(CF)`

#### Collaborative filtering produces recommendations based on the knowledge of users' attitude to otems that is it uses the "wisdom of the crowd" to recomment items

#### Content-based recommender systems foucus on the attributes of the items ans give you recommnedations based on the simmialrity between them.

### In general collabarative filtering is more commonly used that the content-based results and is relatively easy to understand even from overall implementation perspective.

#### The algorithm has the ability to do feature learing on it's own, which means that it can start to learn for iteself what features to use.
* These techniques aim to fill in the missing entries of a user-item association matrix.
* spark.ml currently supports model-based collaborative filtering, in which users and products are described by a small set of latent factors can be used to predict missing entries.
* spark.ml uses the alternating least squares (ALS) algorithm to learn these latent factors.
* The data needs to be in a specifc format to work with Spark's ALS Recommendation Algorithm!

### ALS 
is basically a Matrix Factorization approach to implement a recommendation lgoirthm you decompose you large user/item matrix into lower dimensional user factors and item factor.


### Example

In [9]:
from pyspark.ml.recommendation import ALS

In [10]:
from pyspark.ml.evaluation import RegressionEvaluator

In [11]:
data = spark.read.csv('movielens_ratings.csv', header=True, inferSchema=True)

In [12]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [14]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [15]:
train , test = data.randomSplit([0.8,0.2])

In [16]:
als = ALS(maxIter= 5, regParam=0.01, userCol='userId', itemCol='movieId',ratingCol='rating')

In [17]:
model = als.fit(train)

23/05/21 15:27:00 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/05/21 15:27:00 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/05/21 15:27:02 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [18]:
predictions = model.transform(test)

In [19]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      1|   1.0|     5|  3.2659056|
|      6|   1.0|     4|  1.5368513|
|      6|   1.0|    14|   2.186632|
|      6|   1.0|    18|  0.4537525|
|      3|   2.0|     8|  1.5729034|
|      3|   1.0|    13|  0.9564871|
|      3|   1.0|    26|  0.7006431|
|      3|   1.0|    28|-0.26091012|
|      5|   1.0|    13|  1.6824346|
|      5|   2.0|    18|  2.4998088|
|      5|   1.0|    29|-0.33929783|
|      4|   3.0|     2|   3.816215|
|      4|   2.0|     8|  2.6008193|
|      4|   3.0|    10| 0.27648014|
|      4|   1.0|    12| 0.65203786|
|      2|   3.0|     6|  2.1725338|
|      2|   4.0|     8| -1.4584627|
|      2|   1.0|    23|  2.3096414|
|      0|   1.0|    19| -0.8948506|
|      0|   1.0|    22|  2.2727349|
+-------+------+------+-----------+
only showing top 20 rows



### Evaluation

In [22]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [23]:
rmse = evaluator.evaluate(predictions)

In [24]:
print("RMSE")
print(rmse)

RMSE
1.8164426543965047


#### It's not a good RMSE square

In [26]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [28]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     23|    11|
|     25|    11|
|     27|    11|
|     35|    11|
|     43|    11|
|     45|    11|
|     50|    11|
|     71|    11|
|     77|    11|
|     79|    11|
|     80|    11|
|     88|    11|
|     89|    11|
+-------+------+



In [29]:
recommendations = model.transform(single_user)

In [30]:
recommendations.orderBy('prediction', ascending= False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     27|    11|  5.0584335|
|     23|    11|  2.2466388|
|     25|    11|   1.970675|
|     77|    11|  1.3159499|
|     45|    11| 0.99649704|
|     80|    11|  0.7360893|
|     89|    11|  0.6997621|
|     79|    11|  0.6195072|
|     50|    11|-0.11819631|
|     71|    11| -0.8098814|
|     88|    11| -1.1268233|
|     35|    11| -2.1203852|
|     43|    11| -3.2615855|
+-------+------+-----------+



### Here we can make out user 11 will like movie Id 27 most and 43 least